#import used modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout, LeakyReLU, Bidirectional
from tensorflow.keras.optimizers import Nadam
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import string
import re

#download dataset from kaggle

In [2]:
! chmod 600 kaggle.json && (ls ~/.kaggle 2>/dev/null || mkdir ~/.kaggle) && mv kaggle.json ~/.kaggle/
! kaggle datasets download -d ishantjuyal/emotions-in-text
! ls

  0% 0.00/781k [00:00<?, ?B/s]
100% 781k/781k [00:00<00:00, 52.1MB/s]
emotions-in-text.zip  sample_data


In [3]:
! unzip ./emotions-in-text.zip

Archive:  ./emotions-in-text.zip
  inflating: Emotion_final.csv       


#read data from csv to dataframe

In [ ]:
path = './Emotion_final.csv'
df = pd.read_csv(path, error_bad_lines=False)

#check dataframe

In [ ]:
df.head(5)

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [ ]:
df.isnull().sum()

Text       0
Emotion    0
dtype: int64

In [ ]:
df.shape

(21459, 2)

In [ ]:
print(df['Emotion'].unique())

['sadness' 'anger' 'love' 'surprise' 'fear' 'happy']


In [ ]:
df['Text'].head(10)

0                              i didnt feel humiliated
1    i can go from feeling so hopeless to so damned...
2     im grabbing a minute to post i feel greedy wrong
3    i am ever feeling nostalgic about the fireplac...
4                                 i am feeling grouchy
5    ive been feeling a little burdened lately wasn...
6    ive been taking or milligrams or times recomme...
7    i feel as confused about life as a teenager or...
8    i have been with petronas for years i feel tha...
9                                  i feel romantic too
Name: Text, dtype: object

#preprocessing

In [ ]:
newDf = df.copy()

In [ ]:
newDf

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
21454,Melissa stared at her friend in dism,fear
21455,Successive state elections have seen the gover...,fear
21456,Vincent was irritated but not dismay,fear
21457,Kendall-Hume turned back to face the dismayed ...,fear


In [ ]:
classEncoder = OneHotEncoder(sparse = False)
classColumns = classEncoder.fit_transform(newDf[['Emotion']])
classColumns

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]])

In [ ]:
classColumnsdf = pd.DataFrame(classColumns)
classColumnsdf.columns = [i for i in classEncoder.categories_[0]]
outputNum = len([i for i in classEncoder.categories_[0]])
classColumnsdf

,anger,fear,happy,love,sadness,surprise
0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
21454,0.0,1.0,0.0,0.0,0.0,0.0
21455,0.0,1.0,0.0,0.0,0.0,0.0
21456,0.0,1.0,0.0,0.0,0.0,0.0
21457,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
newDf.drop('Emotion', axis = 1, inplace = True)
newDf = pd.concat([newDf, classColumnsdf], axis = 1)
newDf

,Text,anger,fear,happy,love,sadness,surprise
0,i didnt feel humiliated,0.0,0.0,0.0,0.0,1.0,0.0
1,i can go from feeling so hopeless to so damned...,0.0,0.0,0.0,0.0,1.0,0.0
2,im grabbing a minute to post i feel greedy wrong,1.0,0.0,0.0,0.0,0.0,0.0
3,i am ever feeling nostalgic about the fireplac...,0.0,0.0,0.0,1.0,0.0,0.0
4,i am feeling grouchy,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
21454,Melissa stared at her friend in dism,0.0,1.0,0.0,0.0,0.0,0.0
21455,Successive state elections have seen the gover...,0.0,1.0,0.0,0.0,0.0,0.0
21456,Vincent was irritated but not dismay,0.0,1.0,0.0,0.0,0.0,0.0
21457,Kendall-Hume turned back to face the dismayed ...,0.0,1.0,0.0,0.0,0.0,0.0


#split and process train test data

In [ ]:
tokenizer = Tokenizer(17000, lower = False)
tokenizer.fit_on_texts(newDf['Text'])
numWord = len(tokenizer.word_index)

In [ ]:
numWord

19943

In [ ]:
trainX, validX, trainY, validY = train_test_split(newDf['Text'], newDf.drop('Text', axis=1).values,
                               test_size = 0.2, random_state = 81, stratify = newDf.drop('Text', axis=1).values)

In [ ]:
trainX.size

17167

In [ ]:
testX.size

4292

In [ ]:
def createSequences(texts, tokenizer):
  sequences = tokenizer.texts_to_sequences(texts.values)
  return pad_sequences(sequences=sequences, padding='post')

In [ ]:
trainSeq = createSequences(trainX, tokenizer)
validSeq = createSequences(validX, tokenizer)

In [ ]:
trainSeq

array([[   1,   65,  245, ...,    0,    0,    0],
       [   1,   20,  119, ...,    0,    0,    0],
       [   1,  193,  257, ...,    0,    0,    0],
       ...,
       [   1,    2,  357, ...,    0,    0,    0],
       [   1,  408,    6, ...,    0,    0,    0],
       [   1,   20, 3035, ...,    0,    0,    0]], dtype=int32)

In [ ]:
def defineModel():
  layers = [
            Embedding(numWord, 64),
            Bidirectional(LSTM(128, return_sequences=True)),
            Bidirectional(LSTM(64)),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dense(16, activation='relu'),
            Dense(outputNum, activation='softmax')

  ]
  model = Sequential(layers)
  model.compile(loss = 'categorical_crossentropy', optimizer = Nadam(learning_rate = 0.01), metrics = ['accuracy'])
  return model

In [ ]:
model = defineModel()
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 64)          1276352   
_________________________________________________________________
bidirectional_6 (Bidirection (None, None, 256)         197632    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 128)               164352    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_10 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_11 (Dense)             (None, 6)                

In [ ]:
CHECKPOINT = './checkpoints/checkpoint.ckpt'
earlyStopping = EarlyStopping(monitor = 'loss', 
                              mode = 'min', 
                              patience = 5
)
checkpoint = ModelCheckpoint(filepath = CHECKPOINT,
                             monitor = 'val_accuracy', 
                             save_weights_only=True, 
                             mode = 'max', 
                             save_best_only = True
)
callbacks = [
        earlyStopping,
        checkpoint
]

model.fit(
    x = trainSeq,
    y = trainY,
    epochs = 200,
    steps_per_epoch = 16,
    verbose = 1,
    callbacks = callbacks,
    validation_data = (validSeq, validY),
    batch_size = 128,
    validation_batch_size = 128
)

Epoch 1/200
16/16 [==============================] - 27s 1s/step - loss: 1.6786 - accuracy: 0.3398 - val_loss: 1.6052 - val_accuracy: 0.3276
Epoch 2/200
16/16 [==============================] - 18s 1s/step - loss: 1.6033 - accuracy: 0.3247 - val_loss: 1.5913 - val_accuracy: 0.3276
Epoch 3/200
16/16 [==============================] - 18s 1s/step - loss: 1.6498 - accuracy: 0.3140 - val_loss: 1.5942 - val_accuracy: 0.3276
Epoch 4/200
16/16 [==============================] - 18s 1s/step - loss: 1.5770 - accuracy: 0.3315 - val_loss: 1.5884 - val_accuracy: 0.3276
Epoch 5/200
16/16 [==============================] - 18s 1s/step - loss: 1.5912 - accuracy: 0.3013 - val_loss: 1.5835 - val_accuracy: 0.3276
Epoch 6/200
16/16 [==============================] - 18s 1s/step - loss: 1.6141 - accuracy: 0.2998 - val_loss: 1.5925 - val_accuracy: 0.3276
Epoch 7/200
16/16 [==============================] - 18s 1s/step - loss: 1.5670 - accuracy: 0.3208 - val_loss: 1.4805 - val_accuracy: 0.3276
Epoch 8/200
1

In [ ]:
final_model = defineModel()
model.load_weights(CHECKPOINT)

In [ ]:
loss, acc = model.evaluate(validSeq, validY, verbose = 1)
print(f"final accuracy: {acc * 100}")

135/135 [==============================] - 7s 53ms/step - loss: 0.2716 - accuracy: 0.9150
final accuracy: 91.49580597877502
